In [2]:
import numpy as np
import torch

import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets


torch.cuda.is_available()

True

### GPU HELL YEAH!

In [3]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
        print('Rolling on GPU, babe')
        return device
    print('CPU it is...')
    return torch.device('cpu')

device = get_device()

Rolling on GPU, babe


## Data


In [4]:
# loading data
train = datasets.MNIST("", train=True, download=True,
                       transform=transforms.Compose([transforms.ToTensor()]))
test = datasets.MNIST("", train=False, download=True,
                       transform=transforms.Compose([transforms.ToTensor()]))

In [5]:
trainset = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=128, shuffle=True)

In [12]:
class MLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

net = MLP().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)


In [13]:
# Training the model
EPOCHS = 10
for epoch in range(EPOCHS):
    for data in trainset:
        X, y = data
        X, y = X.to(device), y.to(device)
        net.zero_grad()

        output = net(X.view(-1, 28*28))
        loss = F.nll_loss(output, y)

        loss.backward()
        optimizer.step()

    print(loss)

tensor(0.0653, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1790, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0588, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0577, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0568, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0113, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0271, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0049, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0004, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0192, device='cuda:0', grad_fn=<NllLossBackward0>)


In [14]:
correct = 0
total = 0

with torch.no_grad():
    for data in testset:
        X, y = data
        X, y = X.to(device), y.to(device)
        output = net(X.view(-1,784))
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1
            
print("Accuracy: ", round(correct/total, 3))

Accuracy:  0.981
